In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

from open_hardware_definitions import *
from open_hardware_definitions.common import DATASHEET_DIR, DEFINITIONS_DIR

from py_pdf_parser import tables
from py_pdf_parser.common import BoundingBox
from py_pdf_parser.loaders import load_file
from py_pdf_parser.visualise import visualise

from collections import namedtuple

from pprint import pprint

In [3]:
# Start Device definition

dev = Device(
    manufacturer='NXP',
    part_number='MPC5775',
    architecture="PowerPC",
    bit_width=32,
    endianness=Endianness.BIG
)


In [4]:
# Load with lower char_margin to fix concatenation of columns, and also specify a custom fuzzy orderer

FUZZY_ROUNDING = 5
def fr(val):
    return round(val / FUZZY_ROUNDING) * FUZZY_ROUNDING

orderer = lambda elements: sorted(
    elements, key=lambda elem: (fr(-elem.y0), fr(elem.x0))
)

doc = load_file(os.path.join(DATASHEET_DIR, "MPC5775KRM.pdf"), {'char_margin': 1}, element_ordering=orderer)
content = doc.elements

In [5]:
# Filter out header and footer elements
header_box = BoundingBox(0, 1000, 740, 1000)
footer_box = BoundingBox(0, 1000, 0, 50)
for page in range(1, doc.number_of_pages):
    content -= content.filter_partially_within_bounding_box(header_box, page)
    content -= content.filter_partially_within_bounding_box(footer_box, page)

In [6]:
# Extract module base addresses
modules = []
for start, end in [('Peripheral Bridge 0 slot assignments', 'PMC register base address is 0xFFFA0400'), ('Peripheral Bridge 1 slot assignments', 'MC_CGM memory map')]:
    mod_base_addr_before = content.filter_by_text_contains(start)[0]
    mod_base_addr_after = content.after(mod_base_addr_before).filter_by_text_contains(end)[0]
    mod_base_addr_table_elements = content.between(mod_base_addr_before, mod_base_addr_after).filter_by_fonts("PHUAOA+HelveticaLTStd-Bold,9.0", "PHUAOA+HelveticaLTStd-Roman,9.0")

    mod_base_addr_table_data = mod_base_addr_table_elements.filter_by_font("PHUAOA+HelveticaLTStd-Roman,9.0")[:-1]
    # visualise(doc, 372, elements=mod_base_addr_table_data)

    t = tables.extract_simple_table(mod_base_addr_table_data, as_text=True, allow_gaps=True)

    for d in t:
        if "reserved" in d[4].lower():
            continue

        try:
            name = d[4].split("(")[1].split(")")[0].replace("\n", "")

            # Quirk
            if 'see below' in name:
                name = d[4].split("(")[0].replace("\n", "")
        except IndexError:
            name = d[4].replace("\n", "")

        if len(name.strip()) == 0:
            continue

        base = int(d[0], 16)
        length = int(d[1], 16) - base

        modules.append(Module(
            name=name.strip(),
            base_addr=base,
            size=length,
            description=d[4].replace("\n", ""),
        ))

for m in modules:
    print(m)

Module 'PBRIDGE_0': Base address: 0xfc000000
Module 'XBAR_0': Base address: 0xfc004000
Module 'XBAR_1': Base address: 0xfc008000
Module 'PRAM_XBAR': Base address: 0xfc00c000
Module 'SMPU_0': Base address: 0xfc010000
Module 'SMPU_1': Base address: 0xfc014000
Module 'XBIC_1': Base address: 0xfc018000
Module 'XBIC_0': Base address: 0xfc01c000
Module 'PRAM': Base address: 0xfc020000
Module 'XBIC_2': Base address: 0xfc024000
Module 'PCM': Base address: 0xfc028000
Module 'PFLASH': Base address: 0xfc030000
Module 'XBIC_3': Base address: 0xfc034000
Module 'XBIC_4': Base address: 0xfc038000
Module 'SEMA4': Base address: 0xfc03c000
Module 'INTC0': Base address: 0xfc040000
Module 'XBIC_5': Base address: 0xfc044000
Module 'SWT_0': Base address: 0xfc050000
Module 'SWT_1': Base address: 0xfc054000
Module 'SWT_2': Base address: 0xfc058000
Module 'STM_0': Base address: 0xfc068000
Module 'STM_1': Base address: 0xfc06c000
Module 'STM_2': Base address: 0xfc070000
Module 'EIM': Base address: 0xfc07c000
Mo

In [7]:
# Add regions (manual input for now)
dev.regions = [
    Region("UTest NVM Block", 0x00400000, 0x00404000 - 0x00400000, True, False, False, False),
    Region("EEPROM", 0x00800000, 0x0081800 - 0x00800000, True, True, False, False),
    Region("Flash", 0x00F98000, 0x01380000 - 0x00F98000, True, True, True, False),
    Region("SRAM", 0x40000000, 0x40180000 - 0x40000000, True, True, False, False),
    Region("Local SRAM", 0x50800000, 0x50830000 - 0x50800000, True, True, False, False),
]

In [8]:
# This reference manual looks like a mess, let's only parse the parts that we're interested in

# FlexCAN
registers = []
reg_map_before = content.filter_by_text_equal('CAN memory map')[0]
reg_map_after = content.after(reg_map_before).filter_by_text_contains('Module Configuration Register (CAN_MCR)')[1]
reg_map_elements = content.between(reg_map_before, reg_map_after).filter_by_fonts("PHUAOA+HelveticaLTStd-Roman,9.0")

t = tables.extract_simple_table(reg_map_elements, as_text=True, allow_gaps=True)
for m in modules:
  m.registers = []
  if 'FLEXCAN' in m.name:
    for d in t:
      m.registers.append(Register(
        name=d[1].split('(')[1].split(')')[0].strip(),
        description=d[1].split('(')[0].strip(),
        addr=m.base_addr + int(d[0], 16),
        size_bits=int(d[2], 10),
        read_allowed=('R' in d[3]),
        write_allowed=('W' in d[3]),
        reset_value=int(d[4].replace('_', '').replace('h', ''), 16) if not 'Undefined' in d[4] else None,
      ))


In [9]:
dev.modules = modules

print(dev.dump())

!Device
manufacturer: NXP
part_number: MPC5775
architecture: PowerPC
bit_width: 32
endianness: big
regions:
- !Region
  name: UTest NVM Block
  base_addr: 0x400000
  size: 0x4000
  readable: true
  writable: false
  executable: false
  volatile: false
- !Region
  name: EEPROM
  base_addr: 0x800000
  size: -0x77e800
  readable: true
  writable: true
  executable: false
  volatile: false
- !Region
  name: Flash
  base_addr: 0xf98000
  size: 0x3e8000
  readable: true
  writable: true
  executable: true
  volatile: false
- !Region
  name: SRAM
  base_addr: 0x40000000
  size: 0x180000
  readable: true
  writable: true
  executable: false
  volatile: false
- !Region
  name: Local SRAM
  base_addr: 0x50800000
  size: 0x30000
  readable: true
  writable: true
  executable: false
  volatile: false
modules:
- !Module
  name: PBRIDGE_0
  description: Peripheral Bridge 0 (PBRIDGE_0)
  base_addr: 0xfc000000
  size: 0x3fff
- !Module
  name: XBAR_0
  description: Crossbar 0 (XBAR_0)
  base_addr: 0xfc

In [10]:
# Commit to file
with open(DEFINITIONS_DIR + f"/{dev.manufacturer}/{dev.part_number}.yaml", 'w') as f:
    f.write(dev.dump())